In [58]:
from google.cloud import aiplatform
from vertexai.language_models import TextEmbeddingModel
from tqdm._tqdm_notebook import tqdm_notebook

aiplatform.init(
    project='vidio-quiz-prod',
    location='asia-southeast1',
    staging_bucket='gs://genai_hackathon_2024',
)
tqdm_notebook.pandas()

In [59]:
model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual")

In [81]:
#ganti ke pandas xlsx
import pandas as pd
from datetime import datetime
import numpy as np

df = pd.read_excel('data/film_metadata.xlsx', converters={'release_date':str})
df.head()

/home/tito/.venv/genai_hackathon_2024/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,id,film_title,group_name_l1,group_name_l2,film_main_genre,film_genres,film_directors,film_actors,country_group,description,release_date,total_watchers,age_rating,image_portrait,content_url,image_url
0,9372,Ratu Adil,Series,Vidio Original,crime,"action,crime,drama","ginanti rona,tommy dewo","abdurrahman arif,andri mashadi,budi ros,dian s...",Indonesia,Lasja mendapati suami dan ayahnya terlibat dal...,2024-01-27 00:00:00,393890,18 or more,ratu-adil-57125a.jpg,https://www.vidio.com/premier/9372,https://thumbor.prod.vidiocdn.com/xXSubtNx3jiO...
1,9456,Cinta Pertama Ayah,Series,Vidio Original,crime,"crime,drama,family,mystery",hadrah daeng ratu,"aisha nurra datau,al ghazali,dwi sasono,ersama...",Indonesia,Kehidupan sempurna Amara hancur karena malam p...,2024-01-27 00:00:00,340919,18 or more,cinta-pertama-ayah-66e828.jpg,https://www.vidio.com/premier/9456,https://thumbor.prod.vidiocdn.com/ZONjpNuq_CcE...
2,7617,Bidadari Surgamu,Series,TV Sinetron,drama,"drama,family,religi,romance",anurag vaishnav,"josephine firmstone,michelle joan,rizky nazar,...",Indonesia,Menceritakan kisah seorang gadis bernama Sakin...,2023-03-13 00:00:00,305402,13 or more,bidadari-surgamu-2bbb68.jpg,https://www.vidio.com/premier/7617,https://thumbor.prod.vidiocdn.com/JxL1jUZmTFdc...
3,7576,Magic 5,Series,TV Sinetron,drama,"drama,fantasy,friendship","a. septian,bobby moeryawan,sondang pratama,usm...","afan da5,basmalah,eby da 5,raden rakha,sridevi...",Indonesia,"Naura (Basmalah), Rahsya (Raden Rakha), Adara ...",2023-02-23 00:00:00,263464,13 or more,magic-5-def33f.jpg,https://www.vidio.com/premier/7576,https://thumbor.prod.vidiocdn.com/kha0tcadN-wD...
4,9535,Tertawan Hati,Series,TV Sinetron,drama,"drama,romance",sanjeev ram kishan,"afifah ifah'nda,jonas rivanno,naysilla mirdad,...",Indonesia,Alya dan Alyssa adalah anak kembar yang terpis...,2024-01-22 00:00:00,213724,13 or more,tertawan-hati-592778.jpg,https://www.vidio.com/premier/9535,https://thumbor.prod.vidiocdn.com/sXZL0cyvwRdI...


In [82]:
df.fillna('', inplace=True)
df['title'] = df['film_title'].str.lower()
df['group_l1'] = df['group_name_l1'].str.lower()
df['group_l2'] = df['group_name_l2'].str.lower()
df['genres'] = df['film_genres'].apply(lambda x: ', '.join(x.split(',')))
df['actors'] = df['film_actors'].apply(lambda x: ', '.join(x.split(',')))
df['directors'] = df['film_directors'].apply(lambda x: ', '.join(x.split(',')))
df.loc[df['actors'] == "various", 'actors'] = ""
df['country'] = df['country_group'].str.lower()
df['total_watchers'] = df['total_watchers'].astype('int')
df['release_date'] = df['release_date'].str.replace(" 00:00:00", "")
df['release_year'] = df['release_date'].apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").year if x != '' else '')

def popularity(total_watchers):
    if total_watchers >= 50000:
        return "trending"
    elif total_watchers < 50000 and total_watchers >= 500:
        return "average"
    else:
        return "below average"

df['popularity'] = df['total_watchers'].apply(lambda x: popularity(x))

In [83]:
def search_text(title, description, group_l1, group_l2, main_genre, genres, directors, actors, country, release_year, age_rating, popularity):
    return f"""title: {title}
description: {description}
group: {group_l1} > {group_l2}
genres: {main_genre}, {genres}
directors: {directors}
actors: {actors}
country: {country}
release year: {release_year}
age rating: {age_rating}
popularity: {popularity}"""

In [84]:
search_text_columns = ['title', 'description', 'group_l1', 'group_l2', 'film_main_genre', 'genres', 'directors', 'actors', 'country', 'release_year', 'age_rating', 'popularity']
df['search_text'] = df[search_text_columns].apply(lambda row: search_text(*row), axis=1)
df.head()

,id,film_title,group_name_l1,group_name_l2,film_main_genre,film_genres,film_directors,film_actors,country_group,description,...,title,group_l1,group_l2,genres,actors,directors,country,release_year,popularity,search_text
0,9372,Ratu Adil,Series,Vidio Original,crime,"action,crime,drama","ginanti rona,tommy dewo","abdurrahman arif,andri mashadi,budi ros,dian s...",Indonesia,Lasja mendapati suami dan ayahnya terlibat dal...,...,ratu adil,series,vidio original,"action, crime, drama","abdurrahman arif, andri mashadi, budi ros, dia...","ginanti rona, tommy dewo",indonesia,2024,trending,title: ratu adil\ndescription: Lasja mendapati...
1,9456,Cinta Pertama Ayah,Series,Vidio Original,crime,"crime,drama,family,mystery",hadrah daeng ratu,"aisha nurra datau,al ghazali,dwi sasono,ersama...",Indonesia,Kehidupan sempurna Amara hancur karena malam p...,...,cinta pertama ayah,series,vidio original,"crime, drama, family, mystery","aisha nurra datau, al ghazali, dwi sasono, ers...",hadrah daeng ratu,indonesia,2024,trending,title: cinta pertama ayah\ndescription: Kehidu...
2,7617,Bidadari Surgamu,Series,TV Sinetron,drama,"drama,family,religi,romance",anurag vaishnav,"josephine firmstone,michelle joan,rizky nazar,...",Indonesia,Menceritakan kisah seorang gadis bernama Sakin...,...,bidadari surgamu,series,tv sinetron,"drama, family, religi, romance","josephine firmstone, michelle joan, rizky naza...",anurag vaishnav,indonesia,2023,trending,title: bidadari surgamu\ndescription: Mencerit...
3,7576,Magic 5,Series,TV Sinetron,drama,"drama,fantasy,friendship","a. septian,bobby moeryawan,sondang pratama,usm...","afan da5,basmalah,eby da 5,raden rakha,sridevi...",Indonesia,"Naura (Basmalah), Rahsya (Raden Rakha), Adara ...",...,magic 5,series,tv sinetron,"drama, fantasy, friendship","afan da5, basmalah, eby da 5, raden rakha, sri...","a. septian, bobby moeryawan, sondang pratama, ...",indonesia,2023,trending,"title: magic 5\ndescription: Naura (Basmalah),..."
4,9535,Tertawan Hati,Series,TV Sinetron,drama,"drama,romance",sanjeev ram kishan,"afifah ifah'nda,jonas rivanno,naysilla mirdad,...",Indonesia,Alya dan Alyssa adalah anak kembar yang terpis...,...,tertawan hati,series,tv sinetron,"drama, romance","afifah ifah'nda, jonas rivanno, naysilla mirda...",sanjeev ram kishan,indonesia,2024,trending,title: tertawan hati\ndescription: Alya dan Al...


In [85]:
def embedding_text(model, text):
    embeddings = model.get_embeddings([text])
    for embedding in embeddings:
        vector = embedding.values
        # print(f"Length of Embedding Vector: {len(vector)}")
    return vector

In [86]:
trial_df = df.head()
trial_df['embedding'] = trial_df['search_text'].progress_apply(lambda search_text: embedding_text(model, search_text))

  0%|          | 0/5 [00:00<?, ?it/s]

/tmp/ipykernel_766760/4271889215.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_df['embedding'] = trial_df['search_text'].progress_apply(lambda search_text: embedding_text(model, search_text))


In [87]:
trial_df

,id,film_title,group_name_l1,group_name_l2,film_main_genre,film_genres,film_directors,film_actors,country_group,description,...,group_l1,group_l2,genres,actors,directors,country,release_year,popularity,search_text,embedding
0,9372,Ratu Adil,Series,Vidio Original,crime,"action,crime,drama","ginanti rona,tommy dewo","abdurrahman arif,andri mashadi,budi ros,dian s...",Indonesia,Lasja mendapati suami dan ayahnya terlibat dal...,...,series,vidio original,"action, crime, drama","abdurrahman arif, andri mashadi, budi ros, dia...","ginanti rona, tommy dewo",indonesia,2024,trending,title: ratu adil\ndescription: Lasja mendapati...,"[-0.004612791817635298, -0.009792697615921497,..."
1,9456,Cinta Pertama Ayah,Series,Vidio Original,crime,"crime,drama,family,mystery",hadrah daeng ratu,"aisha nurra datau,al ghazali,dwi sasono,ersama...",Indonesia,Kehidupan sempurna Amara hancur karena malam p...,...,series,vidio original,"crime, drama, family, mystery","aisha nurra datau, al ghazali, dwi sasono, ers...",hadrah daeng ratu,indonesia,2024,trending,title: cinta pertama ayah\ndescription: Kehidu...,"[0.009725026786327362, -0.029527712613344193, ..."
2,7617,Bidadari Surgamu,Series,TV Sinetron,drama,"drama,family,religi,romance",anurag vaishnav,"josephine firmstone,michelle joan,rizky nazar,...",Indonesia,Menceritakan kisah seorang gadis bernama Sakin...,...,series,tv sinetron,"drama, family, religi, romance","josephine firmstone, michelle joan, rizky naza...",anurag vaishnav,indonesia,2023,trending,title: bidadari surgamu\ndescription: Mencerit...,"[0.014840826392173767, -0.0003255524206906557,..."
3,7576,Magic 5,Series,TV Sinetron,drama,"drama,fantasy,friendship","a. septian,bobby moeryawan,sondang pratama,usm...","afan da5,basmalah,eby da 5,raden rakha,sridevi...",Indonesia,"Naura (Basmalah), Rahsya (Raden Rakha), Adara ...",...,series,tv sinetron,"drama, fantasy, friendship","afan da5, basmalah, eby da 5, raden rakha, sri...","a. septian, bobby moeryawan, sondang pratama, ...",indonesia,2023,trending,"title: magic 5\ndescription: Naura (Basmalah),...","[0.051847826689481735, 0.02985983155667782, 0...."
4,9535,Tertawan Hati,Series,TV Sinetron,drama,"drama,romance",sanjeev ram kishan,"afifah ifah'nda,jonas rivanno,naysilla mirdad,...",Indonesia,Alya dan Alyssa adalah anak kembar yang terpis...,...,series,tv sinetron,"drama, romance","afifah ifah'nda, jonas rivanno, naysilla mirda...",sanjeev ram kishan,indonesia,2024,trending,title: tertawan hati\ndescription: Alya dan Al...,"[0.023506544530391693, 0.02385421097278595, -0..."


In [88]:
[print(search_text + "\n") for search_text in trial_df['search_text'].values.tolist()]

title: ratu adil
description: Lasja mendapati suami dan ayahnya terlibat dalam konflik bisnis mafia. Ia bertekad melindungi keluarganya meski harus mengangkat kembali pistolnya.
group: series > vidio original
genres: crime, action, crime, drama
directors: ginanti rona, tommy dewo
actors: abdurrahman arif, andri mashadi, budi ros, dian sastrowardoyo, donny damara, hana malasan, ira wibowo, khiva iskak, muhammad khan, nino fernandez
country: indonesia
release year: 2024
age rating: 18 or more
popularity: trending

title: cinta pertama ayah
description: Kehidupan sempurna Amara hancur karena malam petaka di ulang tahun sahabatnya. Ayahnya, Darma bertekad memperjuangkan keadilannya melalui cara apapun.
group: series > vidio original
genres: crime, crime, drama, family, mystery
directors: hadrah daeng ratu
actors: aisha nurra datau, al ghazali, dwi sasono, ersamayori aurora yatim, rianti cartwright, teuku rifnu wikana, yasmin napper
country: indonesia
release year: 2024
age rating: 18 or mo

[None, None, None, None, None]

In [89]:
# don't run this unless you need it
df['embedding'] = df['search_text'].progress_apply(lambda search_text: embedding_text(model, search_text))

  0%|          | 0/5771 [00:00<?, ?it/s]

In [90]:
final_df = df.loc[:,~df.columns.duplicated()]
final_df.drop(columns=['film_title', 'group_name_l1', 'group_name_l2', 'film_main_genre', 'film_genres', 'film_directors', 'film_actors', 'country_group'], inplace=True)
final_df.head()

,id,description,release_date,total_watchers,age_rating,image_portrait,content_url,image_url,title,group_l1,group_l2,genres,actors,directors,country,release_year,popularity,search_text,embedding
0,9372,Lasja mendapati suami dan ayahnya terlibat dal...,2024-01-27,393890,18 or more,ratu-adil-57125a.jpg,https://www.vidio.com/premier/9372,https://thumbor.prod.vidiocdn.com/xXSubtNx3jiO...,ratu adil,series,vidio original,"action, crime, drama","abdurrahman arif, andri mashadi, budi ros, dia...","ginanti rona, tommy dewo",indonesia,2024,trending,title: ratu adil\ndescription: Lasja mendapati...,"[-0.004612791817635298, -0.009792697615921497,..."
1,9456,Kehidupan sempurna Amara hancur karena malam p...,2024-01-27,340919,18 or more,cinta-pertama-ayah-66e828.jpg,https://www.vidio.com/premier/9456,https://thumbor.prod.vidiocdn.com/ZONjpNuq_CcE...,cinta pertama ayah,series,vidio original,"crime, drama, family, mystery","aisha nurra datau, al ghazali, dwi sasono, ers...",hadrah daeng ratu,indonesia,2024,trending,title: cinta pertama ayah\ndescription: Kehidu...,"[0.009725026786327362, -0.029527712613344193, ..."
2,7617,Menceritakan kisah seorang gadis bernama Sakin...,2023-03-13,305402,13 or more,bidadari-surgamu-2bbb68.jpg,https://www.vidio.com/premier/7617,https://thumbor.prod.vidiocdn.com/JxL1jUZmTFdc...,bidadari surgamu,series,tv sinetron,"drama, family, religi, romance","josephine firmstone, michelle joan, rizky naza...",anurag vaishnav,indonesia,2023,trending,title: bidadari surgamu\ndescription: Mencerit...,"[0.014840826392173767, -0.0003255524206906557,..."
3,7576,"Naura (Basmalah), Rahsya (Raden Rakha), Adara ...",2023-02-23,263464,13 or more,magic-5-def33f.jpg,https://www.vidio.com/premier/7576,https://thumbor.prod.vidiocdn.com/kha0tcadN-wD...,magic 5,series,tv sinetron,"drama, fantasy, friendship","afan da5, basmalah, eby da 5, raden rakha, sri...","a. septian, bobby moeryawan, sondang pratama, ...",indonesia,2023,trending,"title: magic 5\ndescription: Naura (Basmalah),...","[0.051847826689481735, 0.02985983155667782, 0...."
4,9535,Alya dan Alyssa adalah anak kembar yang terpis...,2024-01-22,213724,13 or more,tertawan-hati-592778.jpg,https://www.vidio.com/premier/9535,https://thumbor.prod.vidiocdn.com/sXZL0cyvwRdI...,tertawan hati,series,tv sinetron,"drama, romance","afifah ifah'nda, jonas rivanno, naysilla mirda...",sanjeev ram kishan,indonesia,2024,trending,title: tertawan hati\ndescription: Alya dan Al...,"[0.023506544530391693, 0.02385421097278595, -0..."


In [91]:
final_df.to_json('film_metadata.json', orient='records', lines=True)

In [92]:
from annoy import AnnoyIndex
annoy_index = AnnoyIndex(768, 'dot')
for index, row in final_df.iterrows():
    annoy_index.add_item(row['id'], row['embedding'])
annoy_index.build(10)

True

In [97]:
query_vector = embedding_text(model, "suami-suami masa kini")
results = annoy_index.get_nns_by_vector(query_vector, 10, search_k=-1)
final_df[final_df['id'].isin(results)]

,id,description,release_date,total_watchers,age_rating,image_portrait,content_url,image_url,title,group_l1,group_l2,genres,actors,directors,country,release_year,popularity,search_text,embedding
842,9632,Sering dianggap cowok playboy Delon berjanji u...,2020-08-12,644,13 or more,suamiku-gagal-move-on-4f82e7.jpg,https://www.vidio.com/premier/9632,https://thumbor.prod.vidiocdn.com/ho4wvmMkCiqA...,suamiku gagal move on,movies,ftv,"drama, romance","gita virga, valentino peter",bethul solihin,indonesia,2020,average,title: suamiku gagal move on\ndescription: Ser...,"[0.04193573072552681, 0.01183710154145956, -1...."
1667,5662,Suara Hati Istri - Pengabdian Sepenuh Hati Dib...,2021-10-25,265,13 or more,ada-dua-cinta-di-hati-suamiku-suara-hati-istri...,https://www.vidio.com/premier/5662,https://thumbor.prod.vidiocdn.com/GTpneGsg__B_...,ada dua cinta di hati suamiku,movies,ftv,"drama, romance","dirly dave, disha devina, haviva rifda",nanang istiabudi,indonesia,2021,below average,title: ada dua cinta di hati suamiku\ndescript...,"[0.018973685801029205, -0.0071000754833221436,..."
2249,8277,Suara Hati Istri | Nadya harus menahan pahit k...,2023-01-07,168,13 or more,suamiku-menikahiku-demi-kembali-pada-istrinya-...,https://www.vidio.com/premier/8277,https://thumbor.prod.vidiocdn.com/vUe0P2_oocWZ...,suamiku menikahiku demi kembali pada istrinya,movies,ftv,"drama, romance","arie dwi andika, dea lestari",joe sandjaya,indonesia,2023,below average,title: suamiku menikahiku demi kembali pada is...,"[0.0036754338070750237, -0.013148459605872631,..."
2818,4811,Series ini diadaptasi dari novel asli NovelToo...,2022-03-25,112,18 or more,tuan-suami-yang-kejam-20f193.png,https://www.vidio.com/premier/4811,https://thumbor.prod.vidiocdn.com/abFv0n62V417...,tuan suami yang kejam,series,indonesia,"drama, romance","farah ringpa dara, naufal gani",alif nurul iman,indonesia,2022,below average,title: tuan suami yang kejam\ndescription: Ser...,"[0.023083051666617393, -0.010037703439593315, ..."
2946,3233,Demi sebuah gengsi dan disanjung oleh teman-te...,2020-01-09,102,13 or more,suamiku-bangga-punya-dua-istri-f315b5.jpg,https://www.vidio.com/premier/3233,https://thumbor.prod.vidiocdn.com/GrgC8rjS-k_9...,suamiku bangga punya dua istri,movies,ftv,"drama, family","gita virga, teuku mirza",m. irhasani,indonesia,2020,below average,title: suamiku bangga punya dua istri\ndescrip...,"[0.013825273141264915, 0.008801051415503025, -..."
3311,8313,Suara Hati Istri | Kania (Meriza Febriani) har...,2023-01-03,81,13 or more,suamiku-sudah-menikahiku-tapi-masih-mencintai-...,https://www.vidio.com/premier/8313,https://thumbor.prod.vidiocdn.com/Vbnp-YhjFELt...,suamiku sudah menikahiku tapi masih mencintai ...,movies,ftv,drama,"boy hamzah, meriza febriani",joe sandjaya,indonesia,2023,below average,title: suamiku sudah menikahiku tapi masih men...,"[0.015561746433377266, -0.03273157775402069, 0..."
3366,3310,Mila dan Jery menikah selama 7 tahun dan dikar...,2020-06-16,78,13 or more,masa-masa-covid-mengajarkan-suamiku-arti-perni...,https://www.vidio.com/premier/3310,https://thumbor.prod.vidiocdn.com/DYxfwctRlXK-...,masa-masa covid mengajarkan suamiku arti perni...,movies,ftv,"drama, family","arie dwi andika, gita sinaga",bobby moeryawan,indonesia,2020,below average,title: masa-masa covid mengajarkan suamiku art...,"[0.04512186720967293, -0.013323131017386913, -..."
3470,3300,Menikah itu harus bisa menerima kelebihan dan ...,2020-09-03,71,13 or more,suamiku-butuh-perhatian-lebih-dari-satu-wanita...,https://www.vidio.com/premier/3300,https://thumbor.prod.vidiocdn.com/MouUIuUFOily...,suamiku butuh perhatian lebih dari satu wanita,movies,ftv,"drama, family","haviva rifda, tyas mirasih",joe sandjaya,indonesia,2020,below average,title: suamiku butuh perhatian lebih dari satu...,"[0.0385374054312706, -0.011960454285144806, 0...."
3653,9153,Miko yang berhubungan dibelakang istrinya Nila...,2022-02-09,62,13 or more,apa-tak-cukup-satu-istri-untuk-suamiku-96aa07.jpg,https://www.vidio.com/premi

In [74]:
results = annoy_index.get_nns_by_item(9372, 10, -1)
final_df[final_df['id'].isin(results)]

,id,description,release_date,total_watchers,age_rating,content_url,image_url,title,group_l1,group_l2,genres,actors,directors,country,release_year,popularity,search_text,embedding
0,9372,Lasja mendapati suami dan ayahnya terlibat dal...,2024-01-27,393890,18 or more,https://www.vidio.com/premier/9372,https://thumbor.prod.vidiocdn.com/xXSubtNx3jiO...,ratu adil,series,vidio original,"action, crime, drama","abdurrahman arif, andri mashadi, budi ros, dia...","ginanti rona, tommy dewo",indonesia,2024,trending,title: ratu adil\ndescription: Lasja mendapati...,"[-0.004612791817635298, -0.009792697615921497,..."
1,9456,Kehidupan sempurna Amara hancur karena malam p...,2024-01-27,340919,18 or more,https://www.vidio.com/premier/9456,https://thumbor.prod.vidiocdn.com/ZONjpNuq/CcE...,cinta pertama ayah,series,vidio original,"crime, drama, family, mystery","aisha nurra datau, al ghazali, dwi sasono, ers...",hadrah daeng ratu,indonesia,2024,trending,title: cinta pertama ayah\ndescription: Kehidu...,"[0.009725026786327362, -0.029527712613344193, ..."
6,8803,"Nina memiliki segalanya, suami yang mapan dan ...",2023-10-13,103249,18 or more,https://www.vidio.com/premier/8803,https://thumbor.prod.vidiocdn.com/RNuyNL1GCnuv...,merajut dendam,series,vidio original,"drama, romance","andri mashadi, carissa perusset, laura basuki,...",razka robby ertanto,indonesia,2023,trending,title: merajut dendam\ndescription: Nina memil...,"[-0.0018008917104452848, -0.002155933296307921..."
21,6993,"Luna, seorang penari, memberanikan diri untuk ...",2022-12-15,32571,21 or more,https://www.vidio.com/premier/6993,https://thumbor.prod.vidiocdn.com/BEr90aspJnhp...,"scandal 2: love, sex & revenge",series,vidio original,"drama, mystery, romance, thriller","bizael tanasale, cinta laura, hans de kraker, ...",razka robby ertanto,indonesia,2022,average,"title: scandal 2: love, sex & revenge\ndescrip...","[-0.00023578497348353267, -0.01957436464726925..."
80,4487,"Persahabatan antara Yuda, Ical, Tobi dan Raka ...",2022-03-09,5239,18 or more,https://www.vidio.com/premier/4487,https://thumbor.prod.vidiocdn.com/5n2sLyTcdUZr...,suami-suami masa kini,series,vidio original,"comedy, drama, romance","amanda rigby, aulia sarah, dwi sasono, marcell...",ody c harahap,indonesia,2022,average,title: suami-suami masa kini\ndescription: Per...,"[0.022217445075511932, 0.007138759363442659, 0..."
414,5580,Meninggalnya Rauf membuat seluruh keluarganya ...,2022-07-07,1283,18 or more,https://www.vidio.com/premier/5580,https://thumbor.prod.vidiocdn.com/iXwtULSX1Zz4...,suka duka berduka,series,vidio original,"comedy, drama, family, romance","atiqah hasiholan, ersamayori aurora yatim, jih...","andri cung, nia dinata",indonesia,2022,average,title: suka duka berduka\ndescription: Meningg...,"[0.009404349140822887, 0.010398794896900654, -..."
509,4827,"Dewi, yang telah sukses membangun karir di Kan...",2022-04-08,1049,13 or more,https://www.vidio.com/premier/4827,https://thumbor.prod.vidiocdn.com/17Tj1Ujuax2B...,pulang,series,vidio original,"drama, romance","cok simbara, della dartyan, dwi sasono, ira wi...",sergi sutanto,indonesia,2022,average,"title: pulang\ndescription: Dewi, yang telah s...","[0.011102039366960526, -0.01583855412900448, -..."
1774,5250,"Lina adalah istri dari Rama, namun dibalik sen...",2022-02-17,245,13 or more,https://www.vidio.com/premier/5250,https://thumbor.prod.vidiocdn.com/TO355cY5gOnK...,aku penjarakan suamiku demi anakku,movies,ftv,"drama, romance","firstriana aldila, fitri rachmadina, frederick...",ninos joned,indonesia,2022,below average,title: aku penjarakan suamiku demi anakku\ndes...,"[-0.004982981365174055, 0.019876034930348396, ..."
2068,2947,Acara pertunangan WILDA dan ADNAN kacau balau ...,2021-02-24,190,13 or more,https://www.vidio.com/premier/2947,https://thumbor.prod.vidiocdn.com/z5kQfkvFer87...,melawan hati,movies,ftv,"drama, romance","marsha aruan, omar daniel",ungke kaumbur,indonesia,2021,below average,title: melawan hati\ndescription: Acara pertun...,"[0.0135